In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("/content/Accent Validation Data Updated (2).xlsx")
df.head()

,Call id,Agent id,Agent name,Call Date,Customer Name,Critical to customer,Comment2,Critical to business,Comment,NC,Comment 2
0,158139939.0,289583.0,Omar Khaled Mahmoud Ezz Elarab,2023-10-01 20:38:42,Salwa,NaN,NaN,System Action and documentation\t,Incomplete Update If delayed record was create...,NaN,NaN
1,158308797.0,261800.0,Huda Abdo Zaghloul Raslan,2023-10-02 12:43:44,Mohamed,NaN,NaN,NaN,NaN,NaN,NaN
2,158836735.0,287908.0,Fouad Hamdi Atef Abokoura,2023-10-03 19:36:51,Saleh,NaN,NaN,NaN,NaN,NaN,NaN
3,159385897.0,261800.0,Huda Abdo Zaghloul Raslan,2023-10-05 13:59:28,Abdelrahman,NaN,NaN,NaN,NaN,NaN,NaN
4,159831412.0,194670.0,Mohamed Mansor Hassan Farhan,2023-10-06 21:28:53,Yasmine,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Importing necessary libraries
import pandas as pd

# Reading multiple sheets from an Excel file
sheets_dict = pd.read_excel("/content/Accent Validation Data Updated (2).xlsx", engine="openpyxl", sheet_name=None)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
# for sheet_name, df in sheets_dict.items():
#     print(f"Sheet '{sheet_name}':\n\n")

sheets = ['Normal Calls without escalation', 'Escalated Calls', 'Outages', 'Hard Calls', 'Hitting Calls']

In [5]:
# Accessing individual sheets and displaying their contents
topics_values, call_ids = [], []
for sheet_name, df in sheets_dict.items():
    if sheet_name in sheets:
        print(f"Sheet '{sheet_name}':")
        nan_counts = df["Call Type"].isna().sum()
        print(f"sheet : {sheet_name}\t  nan values {nan_counts}")
        # df["Call Type"] = df["Call Type"].apply(preprocessing)
        # topics_dict[sheet]["value_counts"] = df["Call Type"].value_counts()
        # topics_dict[sheet]["values"] = df["Call Type"].values
        vals = df["Call Type"].tolist()
        print(len(vals))
        topics_values.extend(vals)
        try:
            IDS = df["Call ID"].tolist()
        except KeyError:
            try:
            # print(df.columns)
                IDS = df["Call ID "].tolist()
            except KeyError:
                IDS = df["Call id "].tolist()
        call_ids.extend(IDS)

Sheet 'Normal Calls without escalation':
sheet : Normal Calls without escalation	  nan values 0
98
Sheet 'Escalated Calls':
sheet : Escalated Calls	  nan values 0
33
Sheet 'Outages':
sheet : Outages	  nan values 0
5
Sheet 'Hard Calls':
sheet : Hard Calls	  nan values 0
43
Sheet 'Hitting Calls':
sheet : Hitting Calls	  nan values 0
6


In [6]:
len(topics_values)

185

In [7]:
len(call_ids)

185

In [8]:
len(set(call_ids))

185

In [9]:
result_df = pd.DataFrame(data= {"call_id" : call_ids, "topics": topics_values })
result_df["topics"] = result_df["topics"].apply(lambda x : x.lower().strip())
result_df

,call_id,topics
0,158139939,suspended
1,158308797,wireless request
2,159831412,early renewal
3,160675031,line down-waiting for customer
4,160588653,wireless request
...,...,...
180,158635819,installation follow up
181,159830825,physical instability-waiting for customer -opt...
182,168882684,wrong card & port
183,168670930,installation follow up


In [10]:
df_topics = pd.DataFrame(data= {"topics": topics_values})
# with open("topics.txt", "w") as f:
#     for topic in df_topics["topics"].unique():
#         f.write(f"{topic}\n")
# for i in df_topics["topics"].value_counts().index:


In [11]:
len(df_topics["topics"].unique())

62

In [12]:
category_df = pd.read_excel("/content/Telecom Egypt _Classified_Topics.xlsx")
category_df.head()

,General Category,Specific Topic
0,Billing and Renewal Issues,Billing Inquiry
1,Billing and Renewal Issues,Billing Fraction
2,Billing and Renewal Issues,Billing & Renewal
3,Billing and Renewal Issues,Billing & Renewal- renewal date
4,Billing and Renewal Issues,Early Renewal


In [13]:
category_df.shape

(58, 2)

In [14]:
def preprocessing(x):
    return x.lower().strip()

category_df["General Category"] = category_df["General Category"].apply(lambda x : preprocessing(x))
category_df["Specific Topic"] = category_df["Specific Topic"].apply(lambda x :  preprocessing(x))

In [15]:
duplicate = category_df[category_df.duplicated("Specific Topic")]
duplicate

,General Category,Specific Topic
31,service disruptions,slowness
41,configuration requests,change offering
45,connection issues,unable to obtain ip
48,connection issues,voice problem follow up after sla


In [16]:
category_df.shape

(58, 2)

In [17]:
category_df.drop_duplicates(subset= ["Specific Topic"], inplace = True)
category_df.shape

(54, 2)

In [18]:
category_df.to_excel("category_pruned.xlsx", index = False)

In [19]:
set1 = set(category_df["Specific Topic"].values)
set2 = set(df_topics["topics"].unique())
set1 = {i.lower().strip() for i in set1}
set2 = {i.lower().strip() for i in set2}
diff2 = set2.difference(set1)
diff2

set()

In [20]:
result_df["General Category"] = result_df["topics"].apply(lambda x : category_df[category_df["Specific Topic"] == x]["General Category"].values[0])
result_df

,call_id,topics,General Category
0,158139939,suspended,general inquiries
1,158308797,wireless request,configuration requests
2,159831412,early renewal,billing and renewal issues
3,160675031,line down-waiting for customer,service disruptions
4,160588653,wireless request,configuration requests
...,...,...,...
180,158635819,installation follow up,customer follow-ups
181,159830825,physical instability-waiting for customer -opt...,technical issues
182,168882684,wrong card & port,connection issues
183,168670930,installation follow up,customer follow-ups


In [21]:
result_df.to_excel("call_general_category_mapping.xlsx", index = False)